In [29]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
import openpyxl
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# Navigators

In [3]:
''' NAVIGATOR BETWAY '''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

In [4]:
''' NAVIGATOR 1XBET '''
url_xbet = 'https://ar.1xbet.com/es/live/football'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#xbet = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
xbet = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
xbet.get(url_xbet)
xbet.set_window_size(1920, 1080)

In [5]:
''' NAVIGATOR BETSSON '''
url_bet = 'https://pba.betsson.bet.ar/apuestas-deportivas/futbol'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator

#bets = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bets = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bets.get(url_bet)
bets.maximize_window()

In [7]:
''' NAVIGATOR       20BET LIVE '''
url = 'https://20bet.com/es'
options = Options()
options.headless = False #for not opening the navigator

veinte = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
veinte.get(url)
veinte.maximize_window()

notific = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.align-right.secondary.slidedown-button')))
notific.click()

cookie = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="platform"]/div[2]/div[3]/div/button')))
cookie.click()


In [8]:
''' NAVIGATOR BODOG ''' 

url_bod = 'https://www.bodog.com/es/sports/futbol'

options = Options()

bod =  webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bod.get(url_bod)
bod.maximize_window()

# Functions

In [9]:
def betway():
    df_bway = pd.DataFrame(columns =['teams', 'home', 'tie', 'away'])
    df_bway2 = pd.DataFrame(columns=['teams', 'over', 'goals', 'under', 'site'])
    for j in tqdm(range(30)):
        for i in range(5):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                
                players_bway = elemento[3]
                home = elemento[7]
                tie  = elemento[8]
                away = elemento[9]

                df_bway.loc[len(df_bway.index)] = [players_bway, home, tie, away]
                #print(players_bway, '|         |', f'j: {j}, i: {i} ')

            except:
                pass
    time.sleep(0.5)
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    dropdown.click()
    time.sleep(1.5)
    TotalGoals = bway.find_element(By.CSS_SELECTOR, '[collectionitem="total-goals"]')
    TotalGoals.click()
    
    time.sleep(1.5)
    for j in tqdm(range(20)):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                teams = elemento[3]
                goals = elemento[7]
                under = elemento[10]
                over  = elemento[8]
                df_bway2.loc[len(df_bway2.index)] = [teams, over, goals, under, 'betWay']
                #print(teams, f'||     goals: {goals}, under: {under}, over: {over} ')
            except:
                pass

    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    time.sleep(1.5)
    HomeXaway = bway.find_element(By.CSS_SELECTOR, 'div[collectionitem="win-draw-win"]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway.csv')
    return df_bway

In [43]:
df_betway = betway()
df_betway.head()

100%|██████████| 20/20 [00:01<00:00, 13.43it/s]


,teams,home,tie,away,over,goals,under,site
0,Colon de Santa Fe -Racing Club,"4,50","2,00","2,62","1,50","0,5","2,40",betWay
1,Universidad de Chile Cat. Femenina -Independie...,"2,25","2,70","3,50","1,75","3,5","1,95",betWay
2,Maghrib Fez -OCK Olympique De Khouribga,"1,14","5,75","29,00","2,05","1,5","1,70",betWay


In [41]:
def One_xbet():
    df_xbet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    for k in range(40):
        for j in range(12):
            try:
                elemento =xbet.find_element(By.XPATH, f'/html/body/div[3]/div[1]/div[2]/div/div/div[4]/div/div/div[1]/div/div[{k}]/div[{j}]').text.split('\n')
                teams = elemento[0]+' '+elemento[1]
                if len(elemento)==24:
                    home = elemento[9]
                    tie  = elemento[10]
                    away = elemento[11]

                    over = elemento[15].replace('-', '1')
                    goals = elemento[16].replace('-', '1')
                    under = elemento[17].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under]
                if len(elemento)==26:
                    home = elemento[11]
                    tie  = elemento[12]
                    away = elemento[13]

                    over = elemento[17].replace('-', '1')
                    goals = elemento[18].replace('-', '1')
                    under = elemento[19].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under, '1XBET']

                
                
                #print(elemento[0], elemento[1],'         |', k,'  ', j, '|',len(elemento))
            except:
                pass
    df_xbet = df_xbet[~df_xbet.teams.str.contains('Campeonato')]
    df_xbet.to_csv('CSV\df_xbet.csv')
    return df_xbet

In [64]:
df_xbet = One_xbet()
df_xbet

,teams,home,tie,away,over,goals,under,site
0,Club Atletico Colon Racing Club (Avellaneda),111,26,1.01,2.24,2.5,1.73,1XBET
1,Bard Raptors St. Lawrence Saints,17,10,1.001,2.29,5.5,1.625,1XBET
2,Vassar Brewers Clarkson Golden Knight,10.8,3.81,1.38,2.18,2,1.685,1XBET
3,Union Dutchmen Hobart Statesmen,4.41,2.296,2.496,1.765,1,2.06,1XBET
4,Rensselaer Engineering RIT Tigers,14.8,4.72,1.275,1.78,2,2.035,1XBET
5,Colorado Mesa Mavericks (Women) Colorado Chris...,-,-,-,1.73,4,2.104,1XBET
6,Minnesota-Duluth Bulldogs (Women) Wayne State ...,3.168,2.225,3.168,1.765,1,2.06,1XBET
7,Minot State Beavers (Women) Minnesota Maverick...,3.52,2.02,3.52,2.275,1,1.63,1XBET
8,Sacramento State (Femenino) Eastern Washington...,17,5.7,1.192,1.57,2,2.41,1XBET
9,Northeastern State RiverHawks (Women) Northwes...,16.3,4.79,1.26,1.936,2,1.865,1XBET


In [45]:
def betsson_live():    
    df_bet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    bool=False
    for k in range(30):
        try:
            elemento = bets.find_element(By.CSS_SELECTOR, f'obg-event-row-container[test-id^="f-"]:nth-of-type({k})').text.split('\n')
            #print(elemento[1]+' '+ elemento[2], f'|| len:{len(elemento)}, nth-of-type: {k}')
            
            teams = elemento[1]+' '+elemento[2]
            
            home = elemento[9]
            tie = elemento[11]
            away = elemento[13]
            
            for q in range(len(elemento)):
                if 'más de ' in elemento[q]:
                    over = elemento[q+1]
                    goals = str(float("".join(re.findall(r'\d+', elemento[q+2])))/10)
                    under = elemento[q+3]
                    #print(over, goals, under)
                    break
            while bool:
                if len(elemento)==23:
                    under = elemento[19]
                    goals = float("".join(re.findall(r'\d+', elemento[18])))/10
                    over  = elemento[17]
                
                if len(elemento)==30:
                    under = elemento[18]
                    goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                    over = elemento[16]

                if len(elemento)==35:
                    under = elemento[19]
                    goals = str(float("".join(re.findall(r'\d+', elemento[18])))/10)
                    over = elemento[17]

                if len(elemento) == 21:
                    under = elemento[18]
                    goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                    over = elemento[16]

            df_bet.loc[len(df_bet.index)]  = [teams, home, tie, away, over, goals, under, 'BetSSon']
        except:
            pass
    df_bet.to_csv('CSV\df_bet_live.csv')
    return df_bet

In [46]:
betsson_live()

,teams,home,tie,away,over,goals,under,site
0,Colón de Santa Fe Racing Club,19.00,5.00,1.22,1.51,1.5,2.42,BetSSon
1,Chilangos FC Valle de Xico F.C.,1.17,5.50,15.00,1.22,3.5,3.65,BetSSon
2,Club Halcones de Rayon Novillos Neza,1.22,5.00,13.00,3.75,3.5,1.21,BetSSon
3,Club Universidad de Chile Women CSD Independie...,2.25,2.65,3.50,1.72,3.5,1.95,BetSSon
4,Alianza Lima Women Club Deportivo Lara Women,4.33,3.10,1.80,2.00,2.5,1.70,BetSSon


In [47]:
def bet20_live():
    df_20bet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    elementos = veinte.find_elements(By.CSS_SELECTOR, f'div[class="event-table__row"]')
    i=0
    for partido in elementos:
        elemento = partido.text.split('\n')
        partido.location_once_scrolled_into_view
        #time.sleep(0.15)
        if len(elemento)==17:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 1
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 1')
        if len(elemento)==18:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 2
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 2')
        if len(elemento)==19:
            teams = elemento[2]+' '+elemento[5]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 3
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 3')
        try:
            df_20bet.loc[len(df_20bet.index)] = [teams, home, tie, away, over, goals, under, '20BET']
        except:
            pass
            

        

        
    elementos[0].location_once_scrolled_into_view
    df_20bet.drop_duplicates(inplace=True)
    df_20bet.to_csv('CSV\df_20bet_live.csv')
    return df_20bet

In [48]:
df_bet20 = bet20_live()
df_bet20

,teams,home,tie,away,over,goals,under,site
0,Amora FC GD Estoril Praia,2.25,2.9,3.5,2.05,2.5,1.7,20BET
1,CD Universidad Cesar Vallejo FC Carlos Stein,1.1,6.75,22,2,3,1.7,20BET
2,Estudiantes de Merida Caracas,2.5,2.9,2.8,2.05,2.5,1.65,20BET
3,Maghreb AS de Fes Olympique Club of Khouribga,1.12,6.25,40,2.15,1.5,1.65,20BET
4,Universidad de Chile Independiente Del Valle,2.25,2.75,3.45,1.7,3.5,1.95,20BET
7,Champions FC Academy Af Sporting San Miguelit...,2.05,3.6,2.95,1.75,2.5,1.9,20BET
8,Inter Playa Del Carmen Leviatan FC,1.12,6.75,15,2.05,3.5,1.65,20BET


In [51]:
def bodog_live():
    df_bodog = pd.DataFrame(columns=['teams', 'home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    for j in range(10):
        for k in range(5):
            try:
                elemento = bod.find_element(By.XPATH, f'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[{j}]/sp-coupon[{k}]/sp-multi-markets/section/article').text.split('\n')
                if len(elemento)>10:
                    teams = elemento[2]+' '+elemento[3]+' '+elemento[4]+' '+elemento[5]+' '+elemento[6]+' '+elemento[7]
                    teams = ''.join([i for i in teams if not i.isdigit()]).replace('+  Apuestas', '')
                    numbers = elemento[-5]+' '+elemento[-4]+' '+elemento[-3]+' '+elemento[-2]+' '+elemento[-1]
                    numbers = numbers.split(' ')

                    home=numbers[0]
                    tie = numbers[1]
                    away = numbers[2]

                    goals = numbers[3]
                    over = numbers[4].replace("(", "").replace(")","")
                    under = numbers[6].replace("(", "").replace(")","")


                    df_bodog.loc[len(df_bodog.index)]=[teams, home, tie, away, over, goals, under, 'bodog']
                    #print(teams,'|', numbers,  f'|| {len(elemento)}', 'j',j,'k', k)
            except:
                pass
    df_bodog.to_csv('CSV\df_bodog.csv')
    return df_bodog

In [55]:
df_bod = bodog_live()
df_bod

,teams,home,tie,away,over,goals,under,site
0,Queens Park Arbroath,10.00,1.05,13.00,4.75,2.5,1.167,bodog


In [58]:
def merge(margen=56):
    ''' MAIN DATAFRAME'''
    df_main = pd.DataFrame(columns=['teams_x', 'home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x','site_x', 'teams_matched', 'score', 'teams_y', 'home_y', 'tie_y','away_y', 'over_y', 'goals_y', 'under_y', 'site_y'])
    i=0
    ''' BODOG // 1xBET '''
    try:
        df_xbet = pd.read_csv('CSV\df_xbet.csv') # 1XBET
        df_bodog = pd.read_csv('CSV\df_bodog.csv') # BODOG

        teams_xbet = df_xbet.teams.tolist()

        df_bodog[['teams_matched', 'score']] = df_bodog.teams.apply(lambda x:process.extractOne(x, teams_xbet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bodog = df_bodog[df_bodog.score>margen]

        df_bodog_xbet = pd.merge(df_bodog, df_xbet, left_on='teams_matched', right_on='teams', how='left')
        df_bodog_xbet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)

        df_main=pd.concat([df_main, df_bodog_xbet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape}, {df_bodog_xbet.shape} ')
    except:
        #print('Something went wrong. Check BODOG or 1XBET')
        #print(f'1XBET: {df_xbet.shape}')
        #print(f'BODOG: {df_bodog.shape}')
        pass

    ''' BODOG // BetWAY '''
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv') # BETWAY
        df_bodog = pd.read_csv('CSV\df_bodog.csv') # BODOG

        teams_bway = df_bway.teams.tolist()

        df_bodog[['teams_matched', 'score']] = df_bodog.teams.apply(lambda x:process.extractOne(x, teams_bway, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bodog = df_bodog[df_bodog.score>margen]

        df_bodog_bway = pd.merge(df_bodog, df_bway, left_on='teams_matched', right_on='teams', how='left')
        df_bodog_bway.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)

        df_main=pd.concat([df_main, df_bodog_bway], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
    except:
        #print('Something went wrong. Check BODOG or BETWAY')
        #print(f'BETWAY: {df_bway.shape}')
        #print(f'BODOG: {df_bodog.shape}')
        pass

    ''' BODOG // BETSSON '''
    try:
        df_bet = pd.read_csv('CSV\df_bet_live.csv') #BETSSON
        df_bodog = pd.read_csv('CSV\df_bodog.csv') # BODOG
        
        teams_bet = df_bet.teams.tolist()
        
        df_bodog[['teams_matched', 'score']] = df_bodog.teams.apply(lambda x:process.extractOne(x, teams_bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bodog = df_bodog[df_bodog.score>margen]

        df_bodog_bet = pd.merge(df_bodog, df_bet, left_on='teams_matched', right_on='teams', how='left')
        df_bodog_bet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main=pd.concat([df_main, df_bodog_bet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
    except:
        #print('Something went wrong. Check BODOG or 20BET')
        #print(f'BETSSON: {df_bet.shape}')
        #print(f'BODOG: {df_bodog.shape}')
        pass

    ''' BODOG // 20BET '''
    try:
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv') #20BET
        df_bodog = pd.read_csv('CSV\df_bodog.csv') # BODOG
        
        teams_20bet = df_20bet.teams.tolist()

        df_bodog[['teams_matched', 'score']] = df_bodog.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bodog = df_bodog[df_bodog.score>margen]

        df_bodog_20bet = pd.merge(df_bodog, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_bodog_20bet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)

        df_main=pd.concat([df_main, df_bodog_20bet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
    except:
        #print('Something went wrong. Check BODOG or 20BET')
        # print(f'20BET: {df_20bet.shape}')
        #print(f'BODOG: {df_bodog.shape}')
        pass

    '''1XBET // BETWAY '''
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv') # BETWAY
        df_xbet = pd.read_csv('CSV\df_xbet.csv') # 1XBETOG

        teams_bway = df_bway.teams.tolist()
        
        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_bway, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>margen]

        df_xbet_betway = pd.merge(df_xbet, df_bway, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_betway.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_xbet_betway], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_xbet_betway.to_csv('CSV\TwoWay\df_1xbet_betway.csv')
        #print('1XBET/BETWAY')
        #display(df_xbet_betway)

    except:
        #print('Something went wrong. Check: 1XBET or BETWAY [block 1] ')
        #print(f'1XBET: {df_xbet.shape}')
        #print(f'BETWAY: {df_bway.shape}')
        pass

    '''1XBET // BETSSON '''
    try:
        df_xbet = pd.read_csv('CSV\df_xbet.csv') # 1XBET
        df_bet = pd.read_csv('CSV\df_bet_live.csv') #BETSSON

        teams_bet = df_bet.teams.tolist()

        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>margen]

        df_xbet_betsson = pd.merge(df_xbet, df_bet, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_betsson.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_xbet_betsson], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_xbet_betsson.to_csv('CSV\TwoWay\df_1xbet_betsson.csv')
        #print('1XBET / BETSSON')
        #display(df_xbet_betsson)
    except:
        #print('Something went wrong. Check: 1XBET or BETSSON [block 2]')
        #print(f'1XBET: {df_xbet.shape}')
        #print(f'BETSSON: {df_bet.shape} ')
        pass

    ''' 1XBET // 20BET ''' 
    try:
        df_xbet = pd.read_csv('CSV\df_xbet.csv') # 1XBET
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv') #20BET

        teams_20bet = df_20bet.teams.tolist()

        df_xbet[['teams_matched', 'score']] = df_xbet.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_xbet = df_xbet[df_xbet.score>margen]

        df_xbet_20bet = pd.merge(df_xbet, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_xbet_20bet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_xbet_20bet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_xbet_20bet.to_csv('CSV\TwoWay\df_1xbet_20bet.csv')
        #print('1XBET // 20BET')
        #display(df_xbet_20bet)
    except:
        #print('Something went wrong. Check: 1xBET or 20BET [block 3]')
        #print(f'1XBET: {df_xbet.shape}')
        #print(f'20BET: {df_20bet.shape}')
        pass

    ''' 20BET // BETWAY ''' 
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv') # BETWAY
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv') #20BET
        
        teams_20bet = df_20bet.teams.tolist() 

        df_bway[['teams_matched', 'score']] = df_bway.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer = fuzz.token_sort_ratio)).apply(pd.Series)
        df_bway = df_bway[df_bway.score>margen]

        df_bway_20bet = pd.merge(df_bway, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_bway_20bet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_bway_20bet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_bway_20bet.to_csv('CSV\TwoWay\df_bway_20bet.csv')
        #print('20BET // BETWAY')
        #display(df_bway_20bet)
    except:
        #print('Something went wrong. Check 20BET or BETWAY [block 4]')
        #print(f'20BET: {df_20bet.shape}')
        #print(f'BETWAY: {df_bway.shape}')
        ''' 20BET // BETSSON '''
    try:
        df_20bet = pd.read_csv('CSV\df_20bet_live.csv')
        teams_20bet = df_20bet.teams.tolist()

        df_bet[['teams_matched', 'score']] = df_bet.teams.apply(lambda x:process.extractOne(x, teams_20bet, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bet = df_bet[df_bet.score>margen]        

        df_bet_20bet = pd.merge(df_bet, df_20bet, left_on='teams_matched', right_on='teams', how='left')
        df_bet_20bet.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_bet_20bet], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_bet_20bet.to_csv('CSV\TwoWay\df_betsson_20bet.csv')
        #print('20BET // BETSSON')
        #display(df_bet_20bet)
    except:
        pass
        #print('Something went wrong. Check: BETSSON or 20BET [block 5]')
        #print(f'BETSSON: {df_bet.shape}')
        #print(f'20BET: {df_20bet.shape}')

    ''' BETWAY // BETSSON '''    
    try:
        df_bway = pd.read_csv('CSV\df_bway.csv')
        df_bet = pd.read_csv('CSV\df_bet_live.csv') #BETSSON

        teams_bway = df_bway.teams.tolist()
        
        df_bet[['teams_matched', 'score']] = df_bet.teams.apply(lambda x:process.extractOne(x, teams_bway, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
        df_bet = df_bet[df_bet.score>margen]

        df_bet_betway = pd.merge(df_bet, df_bway, left_on='teams_matched', right_on='teams', how='left')
        df_bet_betway.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main = pd.concat([df_main, df_bet_betway], axis=0)
        i+=1
        print(f'Merge {i}: {df_main.shape} ')
        #df_bet_betway.to_csv('CSV\TwoWay\df_betsson_betway.csv')
        #print('BETWAY // BETSSON')
        #display(df_bet_betway)
    except:
        #print('Something went wrong. Check: BETSSON or BETWAY [block 6]')
        #print(f'BETSSON: {df_bet.shape}')
        #print(f'BETWAY: {df_bway.shape}')
        pass
    columns = [ 'home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x',
        'home_y', 'tie_y', 'away_y', 'over_y', 'goals_y', 'under_y',]
    df_main = df_main[df_main.goals_x == df_main.goals_y]
    
    df_main.to_excel('CSV\df_main_excel.xlsx')
    for col in columns:
        df_main[col] = df_main[col].astype(str).str.replace('.', ',')
    #df_main.drop('index', axis=1, inplace=True)
    df_main.to_csv('CSV\df_main.csv')
    print(f'Filtered to: {df_main.shape} ')
    
    return df_main


In [65]:
df = merge(55)
df[['teams_x', 'over_x', 'under_x', 'goals_x', 'site_x', 'over_y', 'under_y', 'goals_y', 'site_y']]


Merge 1: (2, 18) 
Merge 2: (7, 18) 
Merge 3: (10, 18) 
Merge 4: (14, 18) 
Merge 5: (15, 18) 
Merge 6: (17, 18) 
Filtered to: (5, 18) 


,teams_x,over_x,under_x,goals_x,site_x,over_y,under_y,goals_y,site_y
2,Independiente del Valle (Femenino) Universidad...,"2,01","1,8","3,5",1XBET,"1,72","1,95","3,5",BetSSon
0,MAS Fez Olympique Khouribga,"3,34","1,33","1,5",1XBET,"3,0","1,35","1,5",20BET
1,Independiente del Valle (Femenino) Universidad...,"2,01","1,8","3,5",1XBET,"1,85","1,8","3,5",20BET
2,Amora FC Estoril,"2,26","1,72","3,0",1XBET,"2,05","1,7","3,0",20BET
0,Club Universidad de Chile Women CSD Independie...,"1,72","1,95","3,5",BetSSon,"1,85","1,8","3,5",20BET


In [ ]:
df.index.is_unique

True

In [60]:
def final_function(minutos):
    # TIME STUFF
    target = pd.datetime.now() + pd.to_timedelta(minutos, unit='min')
    target_minute = target.minute
    target_hour = target.hour
    now = pd.datetime.now()

    i=0
    #THREADS
    # Put everything to work
    while target>now:

        
        betway_thread  = threading.Thread(target=betway)
        One_xbet_thread   = threading.Thread(target=One_xbet)
        #betsson  = threading.Thread(target=betsson_live)
        bet20 = threading.Thread(target=bet20_live)
        bodog = threading.Thread(target=bodog_live)
        
        clear_output()
        i+=1
        print(f'Will work till: {target_hour}hs and {target_minute}mins')
        print(f'Iteration number: {i}')
        print('-------')
        print('Functions running')

        betway_thread.start()
        One_xbet_thread.start()
        #betsson.start()
        bet20.start()
        bodog.start()

        betway_thread.join()
        One_xbet_thread.join()
        #betsson.join()
        bet20.join()
        

        print('Scrapers done')
        print('-------')
        print('Merging start...')
        merge()
        print('-------')
        print('Merging finished! All csv files have been updated')
        #time.sleep(5)
        now = pd.datetime.now()
    

In [66]:
final_function(5)

Will work till: 18hs and 18mins
Iteration number: 25
-------
Functions running


100%|██████████| 20/20 [00:01<00:00, 12.47it/s]


Scrapers done
-------
Merging start...
Merge 1: (2, 18) 
Merge 2: (7, 18) 
Merge 3: (11, 18) 
Merge 4: (14, 18) 
Merge 5: (15, 18) 
Merge 6: (17, 18) 
Filtered to: (2, 18) 
-------
Merging finished! All csv files have been updated
